In [562]:
# WETH/USDC
# pool_addr = "0xc6962004f452be9203591991d15f6b388e09e8d0"
# deposit_amount_usd = 1000
# min_price = 2900
# max_price = 3080
# price_assumption = 2990
# is_pair_toggled = False

# pool_addr = "0xc6962004f452be9203591991d15f6b388e09e8d0"
# deposit_amount_usd = 1000
# min_price = 0.0003262236167
# max_price = 0.0003464023971
# price_assumption = 0.00033631300690533197
# is_pair_toggled = True

# WBTC/USDT
# pool_addr = "0x5969efdde3cf5c0d9a88ae51e47d721096a97203"
# deposit_amount_usd = 1000
# min_price = 113694
# max_price = 120727
# price_assumption = 117211
# is_pair_toggled = False

# pool_addr = "0x5969efdde3cf5c0d9a88ae51e47d721096a97203"
# deposit_amount_usd = 1000
# min_price = 0.000008318167136325647
# max_price = 0.000008839594367203463
# price_assumption = 0.00000856515463691587
# is_pair_toggled = True

# WBTC/WETH
pool_addr = "0x2f5e87C9312fa29aed5c179E456625D79015299c"
deposit_amount_usd = 1000
min_price = 35.83
max_price = 38.17
price_assumption = 37
is_pair_toggled = False

In [ ]:
import pandas as pd
from decimal import getcontext
import json
from uniswapv3_calculator import UniswapV3Calculator

# Set high precision for accurate calculations
getcontext().prec = 50

In [564]:
pool_addr = pool_addr.lower()

pool_day_data = pd.read_csv(f'{pool_addr}_pool_day_data.csv')
pool_ticks_data = pd.read_csv(f'{pool_addr}_pool_ticks_data.csv')

with open(f'{pool_addr}_token_data.json', 'r') as f:
    token_data = json.load(f)

if token_data['pool']['id'] != pool_addr:
    raise ValueError(f"Pool address mismatch: {token_data['pool']['id']} != {pool_addr}")

token0_decimal = int(token_data['token0']['decimals'])
token1_decimal = int(token_data['token1']['decimals'])

fee_tier = int(token_data['pool']['feeTier']) / 1_000_000

price_usd_x = float(token_data['token0']['priceUSD'])
price_usd_y = float(token_data['token1']['priceUSD'])

In [565]:
calculator = UniswapV3Calculator(pool_day_data, pool_ticks_data, fee_tier, token0_decimal, token1_decimal, is_pair_toggled)

P = price_assumption
Pl = min_price
Pu = max_price

in_range = Pl <= P <= Pu
print(f"Position In Range: {in_range}")

if not in_range:
    print("❌ Position is out of range!")
    exit()

# Step 1: Calculate token amounts
print(f"\n💰 Step 1: Calculating Token Amounts...")

amount0, amount1, basic_liquidity_delta = calculator.get_tokens_amount_from_deposit_amount_usd(
    P, Pl, Pu, price_usd_x, price_usd_y, deposit_amount_usd
)

print(f"Amount0: {amount0:.8f}")
print(f"Amount1: {amount1:.8f}")

# Step 2: Calculate deltaL
print(f"\n🌊 Step 2: Calculating deltaL...")

corrected_delta_l = calculator.calculate_position_liquidity(amount0, amount1, Pl, Pu, P)

# Step 3: Get total liquidity
print(f"\n📊 Step 3: Getting Total Liquidity...")

current_tick = calculator.get_tick_from_price(P)
print(f"Current tick: {current_tick}")
total_liquidity = calculator.get_liquidity_from_tick(current_tick)
print(f"Total Liquidity (L): {total_liquidity:.2e}")

# Step 4: Calculate fees
print(f"\n💸 Step 4: Calculating Fees...")

volume_24h = calculator.get_volume_24h_avg()
estimated_fee_24h = calculator.estimate_fee(corrected_delta_l, abs(total_liquidity), volume_24h)

monthly_fee = estimated_fee_24h * 30
yearly_fee = estimated_fee_24h * 365

daily_return_pct = (estimated_fee_24h / deposit_amount_usd) * 100
monthly_return_pct = (monthly_fee / deposit_amount_usd) * 100
yearly_apr_pct = (yearly_fee / deposit_amount_usd) * 100

yearly_apr_pct = (yearly_fee / deposit_amount_usd) * 100


print(f"\n✅ Result:")
print(f"Daily Fees: ${estimated_fee_24h:.2f}")
print(f"Monthly Fees: ${monthly_fee:.2f}")
print(f"Yearly Fees: ${yearly_fee:.2f}")
print(f"Yearly APR: {yearly_apr_pct:.2f}%")

Current price (token0/token1): 0.026660288032
Current price (token1/token0): 37.51
Position In Range: True

💰 Step 1: Calculating Token Amounts...
Amount0: 0.00423473
Amount1: 0.16167972

🌊 Step 2: Calculating deltaL...
deltaL: 1.67e+13

📊 Step 3: Getting Total Liquidity...
Current tick: 266381
Total Liquidity (L): 3.25e+16

💸 Step 4: Calculating Fees...
Average 24h volume (last 7 days): $69,353,854.16

✅ Result:
Daily Fees: $17.80
Monthly Fees: $534.10
Yearly Fees: $6498.25
Yearly APR: 649.82%
